In [13]:
from pandas import DataFrame, read_csv
import pandas as pd
import json as js
import os

pl_file = os.path.join('generated_data','players_complete.csv')
pl_data = pd.read_csv(pl_file, encoding='ISO-8859-1')
pl_data.head()

,Player ID,SN,Age,Gender
0,0,Marughi89,21,Male
1,1,Lirtedy26,40,Male
2,2,Chamistast30,7,Male
3,3,Lisirra25,24,Male
4,4,Lirtim36,23,Male


In [14]:
i_file = os.path.join('generated_data', 'items_complete.csv')
i_data = pd.read_csv(i_file, encoding='ISO-8859-1')
i_data.head()

,Item ID,Item Name,Price
0,0,Splinter,1.89
1,1,Crucifer,3.67
2,2,Verdict,2.65
3,3,Phantomlight,4.15
4,4,Bloodlord's Fetish,1.91


In [15]:
# read csv purchase data file
ps_file = os.path.join('generated_data', 'purchase_data_3.csv')
ps_data = pd.read_csv(ps_file, encoding='ISO-8859-1')
ps_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Iloni35,20,Male,93,Apocalyptic Battlescythe,4.49
1,1,Aidaira26,21,Male,12,Dawne,3.36
2,2,Irim47,17,Male,5,Putrid Fan,2.63
3,3,Irith83,17,Male,123,Twilight's Carver,2.55
4,4,Philodil43,22,Male,154,Feral Katana,4.11


In [28]:
# read json purchase data file
json_file = os.path.join('generated_data', 'purchase_data.json')
with open(json_file, 'r') as jdata:
    jdata_s = js.load(jdata)
jdata_df = pd.DataFrame(jdata_s)
jdata_df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [29]:
# total player count
pl_total = pl_data['Player ID'].count()
pl_total

1163

In [37]:
# purchasing analysis function
def purchase_analysis(dataframe):
    unique = len(dataframe['Item ID'].unique())
    ps_avg = dataframe['Price'].mean()
    ps_con = dataframe['Item ID'].count()
    ps_sum = dataframe['Price'].sum()
    ps_out_df = pd.DataFrame({'Unique Items':[unique],
                            'Purchase Avg.':[ps_avg],
                            'Purchase Count':[ps_con],
                            'Total Revenue':[ps_sum]})
    ps_out_df['Purchase Avg.'] = ps_out_df['Purchase Avg.'].map('${:.2f}'.format)
    ps_out_df['Total Revenue'] = ps_out_df['Total Revenue'].map('${:.2f}'.format)
    return ps_out_df

In [38]:
# csv file analysis
purchase_analysis(ps_data).head()

,Purchase Avg.,Purchase Count,Total Revenue,Unique Items
0,$2.92,78,$228.10,64


In [39]:
# json file analysis
purchase_analysis(jdata_df).head()

,Purchase Avg.,Purchase Count,Total Revenue,Unique Items
0,$2.93,780,$2286.33,183


In [60]:
# Gender Demographics function
def gender_analysis (dataframe):
    g_df = pd.DataFrame(dataframe['Item ID'].groupby(dataframe['Gender']).count()).rename(columns={'Item ID':'Count'})
    g_p_df = pd.DataFrame(dataframe['Price'].groupby(dataframe['Gender']).sum()).rename(columns={'Price':'Revenue'})
    g_avg = g_p_df['Revenue']/g_df['Count']
    g_df['Purchase Avg.'] = g_avg.map('${:,.2f}'.format)
    g_df['Revenue'] = g_p_df['Revenue'].map('${:,.2f}'.format)
    return g_df

In [61]:
# gender analysis csv file
gender_analysis(ps_data)

,Count,Purchase Avg.,Revenue
Gender,,,
Female,13,$3.18,$41.38
Male,64,$2.88,$184.60
Other / Non-Disclosed,1,$2.12,$2.12


In [62]:
# gender analysi json file
gender_analysis(jdata_df)

,Count,Purchase Avg.,Revenue
Gender,,,
Female,136,$2.82,$382.91
Male,633,$2.95,"$1,867.68"
Other / Non-Disclosed,11,$3.25,$35.74


In [76]:
# Age Demographics function
def age_analysis(dataframe):
    bins = [0,10,15,25,35,45]
    groups = ['Child','Junior','Youth','Adult','Old']
    a_data = dataframe
    a_data['Age Category'] = pd.cut(a_data['Age'], bins, labels=groups)
    a_rev_df = pd.DataFrame(a_data['Price'].groupby(a_data['Age Category']).sum()).rename(columns={'Price':'Revenue'})
    a_df = pd.DataFrame(a_data['Item ID'].groupby(a_data['Age Category']).count()).rename(columns={'Item ID':'Count'})
    a_df['Purchase Avg.'] = a_rev_df['Revenue']/a_df['Count']
    a_df['Purchase Avg.'] = a_df['Purchase Avg.'].map('${:,.2f}'.format)
    a_df['Revenue'] = a_rev_df['Revenue'].map('${:,.2f}'.format)
    return a_df

In [77]:
# age demographics csv file
age_analysis(ps_data)

,Count,Purchase Avg.,Revenue
Age Category,,,
Child,5,$2.76,$13.82
Junior,4,$3.05,$12.21
Youth,53,$2.93,$155.11
Adult,11,$2.48,$27.24
Old,5,$3.94,$19.72


In [78]:
# age demographics json file
age_analysis(jdata_df)

,Count,Purchase Avg.,Revenue
Age Category,,,
Child,32,$3.02,$96.62
Junior,78,$2.87,$224.15
Youth,489,$2.93,"$1,431.35"
Adult,134,$2.97,$398.08
Old,47,$2.90,$136.13


In [90]:
# Top 5 spenders function
def top_five_players(dataframe):
    SN_df = pd.DataFrame(dataframe['Item ID'].groupby(dataframe['SN']).count()).rename(columns={'Item ID':'Count'})
    SN_rev_df = pd.DataFrame(dataframe['Price'].groupby(dataframe['SN']).sum()).rename(columns={'Price':'Revenue'})
    SN_avg_df = SN_rev_df['Revenue']/SN_df['Count']
    SN_df['Purchase Avg.'] = SN_avg_df.map('${:.2f}'.format)
    SN_df['Revenue'] = SN_rev_df['Revenue'].map('${:.2f}'.format)
    sorted_df = SN_df.sort_values('Revenue', ascending=False).head()
    return sorted_df

In [91]:
# Top 5 from csv file
top_five_players(ps_data)

,Count,Purchase Avg.,Revenue
SN,,,
Sundaky74,2,$3.71,$7.41
Aidaira26,2,$2.56,$5.13
Eusty71,1,$4.81,$4.81
Chanirra64,1,$4.78,$4.78
Alarap40,1,$4.71,$4.71


In [98]:
# top 5 from json file
top_five_players(jdata_df)

,Count,Purchase Avg.,Revenue
SN,,,
Qarwen67,4,$2.49,$9.97
Sondim43,3,$3.13,$9.38
Tillyrin30,3,$3.06,$9.19
Lisistaya47,3,$3.06,$9.19
Tyisriphos58,2,$4.59,$9.18


In [99]:
# top five items function
def top_five_items(dataframe):
    i_df = pd.DataFrame(dataframe['Item ID'].groupby([dataframe['Item ID'], dataframe['Item Name']]).count()).rename(columns={'Item ID':'Count'})
    i_rev_df = pd.DataFrame(dataframe['Price'].groupby([dataframe['Item ID'], dataframe['Item Name']]).sum()).rename(columns={'Price':'Revenue'})
    i_p_df = i_rev_df['Revenue']/i_df['Count']
    i_df['Price'] = i_p_df.map('${:.2f}'.format)
    i_df['Revenue'] = i_rev_df['Revenue']
    i_table = i_df.sort_values('Revenue', ascending=False).head()
    i_table['Revenue'] = i_table['Revenue'].map('${:.2f}'.format)
    return i_table

In [100]:
# top 5 items in csv file
top_five_items(ps_data)

,,Count,Price,Revenue
Item ID,Item Name,,,
94,Mourning Blade,3,$3.64,$10.92
117,"Heartstriker, Legacy of the Light",2,$4.71,$9.42
93,Apocalyptic Battlescythe,2,$4.49,$8.98
90,Betrayer,2,$4.12,$8.24
154,Feral Katana,2,$4.11,$8.22


In [101]:
# top 5 items in json file
top_five_items(jdata_df)

,,Count,Price,Revenue
Item ID,Item Name,,,
34,Retribution Axe,9,$4.14,$37.26
115,Spectral Diamond Doomblade,7,$4.25,$29.75
32,Orenmir,6,$4.95,$29.70
103,Singed Scalpel,6,$4.87,$29.22
107,"Splitter, Foe Of Subtlety",8,$3.61,$28.88
